<a href="https://colab.research.google.com/github/shubham8garg/MCP_A2A_session/blob/main/A2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import os

os.environ["OPENAI_API_KEY"] = "sk-abc"
# Verify
assert os.environ.get("OPENAI_API_KEY", "").startswith("sk-"), "❌ Please set a valid OPENAI_API_KEY"
print("✅ OPENAI_API_KEY is set")

✅ OPENAI_API_KEY is set


In [49]:
# Cell 4: Pretty printing helpers (used across all parts)

def print_header(text):
    """Print a prominent section header"""
    width = 60
    print("\n" + "═" * width)
    print(f"  {text}")
    print("═" * width)

def print_step(num, text):
    """Print a numbered step"""
    print(f"\n▶ Step {num}: {text}")
    print(f"{'─' * 55}")

def print_tool_call(name, args):
    """Print a tool call with its arguments"""
    print(f"  📤 TOOL CALL: {name}")
    if isinstance(args, str):
        args = json.loads(args)
    for k, v in args.items():
        print(f"     {k}: {v}")

def print_tool_result(text, max_lines=8):
    """Print a preview of tool results"""
    lines = text.split("\n")
    for line in lines[:max_lines]:
        print(f"  📥 {line}")
    if len(lines) > max_lines:
        print(f"  📥 ... ({len(lines) - max_lines} more lines)")

print("✅ Helper functions defined")

✅ Helper functions defined


---
# 📖 PART 4–7: Agent-to-Agent (A2A) Protocol

## What we'll build:

| Part | What | Goal |
|------|------|------|
| **Part 4** | Build & use a simple A2A agent | Understand A2A basics |
| **Part 5** | A2A agent with LLM + MCP inside | Agents can be smart internally |
| **Part 6** | Multi-agent discovery & delegation | Agents find and use each other |
| **Part 7** | Grand Finale: LLM + MCP + A2A | Full agentic system |

## Key Concept: MCP vs A2A


In [11]:
!pip install openai mcp fastmcp httpx python-dotenv a2a-sdk uvicorn httpx pyngrok -q

In [50]:
# Cell A4: Import everything we'll need for A2A

import subprocess, sys, os, signal, time, json, asyncio
import httpx
from openai import OpenAI

# A2A Types
from a2a.types import (
    AgentCard,
    AgentSkill,
    AgentCapabilities,
    Message,
    TextPart,
    DataPart,
    Part,
    Task,
    TaskState,
    TaskStatus,
    Artifact,
)

# A2A Server
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore

# A2A Client
from a2a.client import A2AClient, A2ACardResolver

# OpenAI (already set up from earlier)
client_openai = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL = "gpt-5.2"

print("✅ All A2A imports successful!")

✅ All A2A imports successful!


In [51]:
# Cell A5: Helpers using the PROPER a2a-sdk API

import uuid

# Additional A2A types we need for sending
from a2a.types import (
    SendMessageRequest,
    SendMessageResponse,
    MessageSendParams,
)

AGENT_PROCESSES = {}

def start_agent_server(name, script_path, port):
    """Start an A2A agent server as a background subprocess."""
    stop_agent_server(name)

    abs_path = os.path.abspath(script_path)
    proc = subprocess.Popen(
        [sys.executable, abs_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
    )
    time.sleep(3)

    if proc.poll() is not None:
        _, stderr = proc.communicate()
        print(f"❌ {name} crashed on startup!")
        print(stderr.decode()[-500:])
        return False

    AGENT_PROCESSES[name] = proc
    print(f"✅ {name} running on port {port} (PID: {proc.pid})")
    return True


def stop_agent_server(name):
    if name in AGENT_PROCESSES:
        proc = AGENT_PROCESSES[name]
        try:
            proc.terminate()
            proc.wait(timeout=5)
        except:
            proc.kill()
        del AGENT_PROCESSES[name]
        print(f"🛑 {name} stopped")


def stop_all_agents():
    for name in list(AGENT_PROCESSES.keys()):
        stop_agent_server(name)
    print("🛑 All agents stopped")


def extract_text_from_message(message):
    """Extract text from an A2A Message object."""
    texts = []
    if message and hasattr(message, 'parts') and message.parts:
        for part in message.parts:
            if hasattr(part, 'root') and hasattr(part.root, 'text'):
                texts.append(part.root.text)
            elif hasattr(part, 'text'):
                texts.append(part.text)
    return "\n".join(texts)


def extract_text_from_response(resp):
    """Extract agent's text from a SendMessageResponse."""
    texts = []
    try:
        # resp is SendMessageResponse which has .root
        # .root is SendMessageSuccessResponse which has .result
        # .result is a Task or Message
        result_obj = resp.root.result

        # If it's a Task, check history and artifacts
        if hasattr(result_obj, 'history') and result_obj.history:
            for msg in result_obj.history:
                if hasattr(msg, 'role') and str(msg.role) != 'user':
                    text = extract_text_from_message(msg)
                    if text:
                        texts.append(text)

        if hasattr(result_obj, 'artifacts') and result_obj.artifacts:
            for art in result_obj.artifacts:
                if hasattr(art, 'parts'):
                    for part in art.parts:
                        if hasattr(part, 'root') and hasattr(part.root, 'text'):
                            texts.append(part.root.text)

        # If it's a Message directly
        if hasattr(result_obj, 'parts'):
            text = extract_text_from_message(result_obj)
            if text:
                texts.append(text)

        # Check status
        if hasattr(result_obj, 'status') and result_obj.status:
            if hasattr(result_obj.status, 'message') and result_obj.status.message:
                status_text = extract_text_from_message(result_obj.status.message)
                if status_text and status_text not in texts:
                    texts.append(status_text)

    except Exception as e:
        texts.append(f"(Parse error: {e})")

    return "\n".join(texts) if texts else str(resp)[:300]


async def send_task_to_agent(base_url, user_text):
    """
    Send a message to an A2A agent using the proper SDK API.

    Chain: SendMessageRequest → MessageSendParams → Message → Parts
    """
    async with httpx.AsyncClient(timeout=30.0) as httpx_client:
        # 1. Resolve agent card
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=base_url)
        card = await resolver.get_agent_card()

        # 2. Create A2A client
        a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)

        # 3. Build the proper request chain
        #    Message → MessageSendParams → SendMessageRequest
        message = Message(
            message_id=str(uuid.uuid4()),
            role="user",
            parts=[Part(root=TextPart(text=user_text))],
        )

        params = MessageSendParams(message=message)

        request = SendMessageRequest(
            id=str(uuid.uuid4()),
            params=params,
        )

        # 4. Send via the official SDK method
        print(f"  📤 Sending: \"{user_text}\"")
        response = await a2a_client.send_message(request=request)

        print(f"  📥 Response received!")
        return response


print("✅ All helpers defined (using proper a2a-sdk API)")
print()
print("📐 Request chain:")
print("   SendMessageRequest(id, params)")
print("     └── MessageSendParams(message)")
print("          └── Message(message_id, role, parts)")
print("               └── Part(root=TextPart(text=...))")

✅ All helpers defined (using proper a2a-sdk API)

📐 Request chain:
   SendMessageRequest(id, params)
     └── MessageSendParams(message)
          └── Message(message_id, role, parts)
               └── Part(root=TextPart(text=...))


# 📖 PART 4: Build Your First A2A Agent

**What is A2A?**
- **Agent-to-Agent Protocol** — an open standard by Google
- Enables agents to **discover**, **communicate**, and **collaborate**
- Each agent publishes an **Agent Card** (like a business card)
- Communication via **Tasks** with **Messages** and **Artifacts**

**Key Components:**
| Component | What | Analogy |
|-----------|------|---------|
| **Agent Card** | JSON describing capabilities | Business card |
| **Skill** | A declared capability | Job title on the card |
| **Task** | A unit of work | A support ticket |
| **Message** | Communication within task | Chat messages on ticket |
| **Artifact** | Structured output | Deliverable / attachment |

**Architecture:**

In [6]:
# Cell A7: Create directory for A2A agents
os.makedirs("a2a_agents", exist_ok=True)
print("✅ Created a2a_agents/ directory")

✅ Created a2a_agents/ directory


In [7]:
%%writefile a2a_agents/currency_agent.py
"""
A2A Agent: Currency Converter
"""

import json
import re
import uuid
import uvicorn

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCard, AgentSkill, AgentCapabilities,
    Message, TextPart, Part,
)


EXCHANGE_RATES = {
    "USD": 1.0, "JPY": 149.50, "EUR": 0.92, "GBP": 0.79,
    "INR": 83.50, "AUD": 1.53, "CAD": 1.36,
}


def convert_currency(amount, from_curr, to_curr):
    from_curr = from_curr.upper()
    to_curr = to_curr.upper()
    if from_curr not in EXCHANGE_RATES or to_curr not in EXCHANGE_RATES:
        return None, f"Unknown currency. Supported: {list(EXCHANGE_RATES.keys())}"
    usd_amount = amount / EXCHANGE_RATES[from_curr]
    result = usd_amount * EXCHANGE_RATES[to_curr]
    return result, None


def parse_conversion_request(text):
    text = text.upper().replace(",", "")
    pattern = r"(\d+(?:\.\d+)?)\s*([A-Z]{3})\s*(?:TO|->|=>|IN)\s*([A-Z]{3})"
    match = re.search(pattern, text)
    if match:
        return float(match.group(1)), match.group(2), match.group(3)
    return None, None, None


class CurrencyAgentExecutor(AgentExecutor):

    async def execute(self, context: RequestContext, event_queue: EventQueue):
        # Extract text from message parts
        user_text = ""
        if context.message and context.message.parts:
            for part in context.message.parts:
                if hasattr(part, "root") and hasattr(part.root, "text"):
                    user_text += part.root.text
                elif hasattr(part, "text"):
                    user_text += part.text

        # Parse and convert
        amount, from_curr, to_curr = parse_conversion_request(user_text)

        if amount is None:
            response_text = (
                "I can convert currencies! Try: '100 USD to JPY'\n"
                f"Supported currencies: {', '.join(EXCHANGE_RATES.keys())}"
            )
        else:
            result, error = convert_currency(amount, from_curr, to_curr)
            if error:
                response_text = f"Error: {error}"
            else:
                rate = round(EXCHANGE_RATES[to_curr] / EXCHANGE_RATES[from_curr], 4)
                response_text = (
                    f"{amount:,.2f} {from_curr} = {result:,.2f} {to_curr}\n"
                    f"(Rate: 1 {from_curr} = {rate} {to_curr})"
                )

        # ⭐ THE FIX: await the enqueue_event call!
        await event_queue.enqueue_event(
            Message(
                message_id=str(uuid.uuid4()),
                role="agent",
                parts=[Part(root=TextPart(text=response_text))],
            )
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue):
        pass


agent_card = AgentCard(
    name="Currency Converter Agent",
    description="Converts between currencies using real-time exchange rates.",
    url="http://localhost:5001",
    version="1.0.0",
    defaultInputModes=["text"],
    defaultOutputModes=["text"],
    skills=[
        AgentSkill(
            id="currency-conversion",
            name="Currency Conversion",
            description="Convert an amount from one currency to another. Example: '100 USD to JPY'",
            tags=["currency", "finance", "conversion"],
            examples=["100 USD to JPY", "500 EUR to GBP", "1000 INR to USD"],
        )
    ],
    capabilities=AgentCapabilities(streaming=False),
)

executor = CurrencyAgentExecutor()
task_store = InMemoryTaskStore()
handler = DefaultRequestHandler(agent_executor=executor, task_store=task_store)
app = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)

if __name__ == "__main__":
    print("Currency Converter Agent starting on port 5001...")
    uvicorn.run(app.build(), host="0.0.0.0", port=5001)

Writing a2a_agents/currency_agent.py


In [52]:
# Cell A9: Verify the agent script has no import errors

result = subprocess.run(
    [sys.executable, "-c", "exec(open('a2a_agents/currency_agent.py').read().split('if __name__')[0])"],
    capture_output=True, text=True, timeout=10,
    env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
)

if result.returncode == 0:
    print("✅ currency_agent.py — No import/syntax errors!")
else:
    print(f"❌ Error:\n{result.stderr}")

✅ currency_agent.py — No import/syntax errors!


In [53]:
# Cell A10: Start the Currency Agent as a background process

success = start_agent_server(
    name="currency-agent",
    script_path="a2a_agents/currency_agent.py",
    port=5001
)

if success:
    print("\n📋 Agent Card URL: http://localhost:5001/.well-known/agent.json")
    print("📮 Task Endpoint:  http://localhost:5001/ (JSON-RPC)")

✅ currency-agent running on port 5001 (PID: 4913)

📋 Agent Card URL: http://localhost:5001/.well-known/agent.json
📮 Task Endpoint:  http://localhost:5001/ (JSON-RPC)


In [12]:
# Paste your ngrok authtoken here:
# You can find it at: https://dashboard.ngrok.com/get-started/your-authtoken
AUTH_TOKEN = "39e3rgNakBDqcl8i3CGAKMU8Hsc_3drNDoarTnAuURP3mDbXp" # <---- REPLACE WITH YOUR TOKEN!

if AUTH_TOKEN == "YOUR_AUTHTOKEN":
    print("❌ Please replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken.")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(AUTH_TOKEN)
    print("✅ pyngrok authtoken configured. Now, please re-run the cell below (Step 2) to establish the tunnel.")

✅ pyngrok authtoken configured. Now, please re-run the cell below (Step 2) to establish the tunnel.


In [22]:
!ngrok authtoken 39e3rgNakBDqcl8i3CGAKMU8Hsc_3drNDoarTnAuURP3mDbXp

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [29]:
print_step(1, "Install pyngrok")
!pip install pyngrok -q


▶ Step 1: Install pyngrok
───────────────────────────────────────────────────────


In [13]:
print_step(2, "Expose port 5001 using pyngrok")
from pyngrok import ngrok
import asyncio

# Terminate any existing ngrok tunnels
ngrok.kill()

try:
    # Open a ngrok tunnel to port 5001
    public_url_object = ngrok.connect(5001)
    public_url = public_url_object.public_url

    if public_url:
        print(f"✅ ngrok tunnel established: {public_url}")
        print(f"You can now access your agent at: {public_url}/.well-known/agent.json from your local machine.")
        NGROK_PUBLIC_URL = public_url
    else:
        print("❌ Failed to get ngrok public URL after connection. Check pyngrok logs.")
except Exception as e:
    print(f"❌ An error occurred while trying to connect ngrok: {e}")
    print("Please ensure the ngrok authtoken is correctly set and try again.")



▶ Step 2: Expose port 5001 using pyngrok
───────────────────────────────────────────────────────
✅ ngrok tunnel established: https://provocative-splurgily-kelvin.ngrok-free.dev
You can now access your agent at: https://provocative-splurgily-kelvin.ngrok-free.dev/.well-known/agent.json from your local machine.


In [14]:
print_step(3, "Send a test message to the Currency Agent via ngrok tunnel")

if 'NGROK_PUBLIC_URL' in globals():
    async def test_currency_agent():
        agent_url = NGROK_PUBLIC_URL
        print(f"Using agent URL: {agent_url}")

        print_tool_call("send_task_to_agent", {"base_url": agent_url, "user_text": "100 USD to JPY"})
        response = await send_task_to_agent(agent_url, "100 USD to JPY")
        print_tool_result(extract_text_from_response(response))

        print_tool_call("send_task_to_agent", {"base_url": agent_url, "user_text": "50 EUR to GBP"})
        response = await send_task_to_agent(agent_url, "50 EUR to GBP")
        print_tool_result(extract_text_from_response(response))

        print_tool_call("send_task_to_agent", {"base_url": agent_url, "user_text": "What is the weather like?"})
        response = await send_task_to_agent(agent_url, "What is the weather like?")
        print_tool_result(extract_text_from_response(response))

    await test_currency_agent()
else:
    print("❌ NGROK_PUBLIC_URL not set. Please run the previous cell first.")


▶ Step 3: Send a test message to the Currency Agent via ngrok tunnel
───────────────────────────────────────────────────────
Using agent URL: https://provocative-splurgily-kelvin.ngrok-free.dev
  📤 TOOL CALL: send_task_to_agent
     base_url: https://provocative-splurgily-kelvin.ngrok-free.dev
     user_text: 100 USD to JPY


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  📤 Sending: "100 USD to JPY"
  📥 Response received!
  📥 100.00 USD = 14,950.00 JPY
  📥 (Rate: 1 USD = 149.5 JPY)
  📤 TOOL CALL: send_task_to_agent
     base_url: https://provocative-splurgily-kelvin.ngrok-free.dev
     user_text: 50 EUR to GBP
  📤 Sending: "50 EUR to GBP"
  📥 Response received!
  📥 50.00 EUR = 42.93 GBP
  📥 (Rate: 1 EUR = 0.8587 GBP)
  📤 TOOL CALL: send_task_to_agent
     base_url: https://provocative-splurgily-kelvin.ngrok-free.dev
     user_text: What is the weather like?
  📤 Sending: "What is the weather like?"
  📥 Response received!
  📥 I can convert currencies! Try: '100 USD to JPY'
  📥 Supported currencies: USD, JPY, EUR, GBP, INR, AUD, CAD


In [54]:
# Cell A11: Discover the agent using A2A Client — read its "business card"

print_header("Discovering Agent via Agent Card")

async def discover_agent(base_url):
    async with httpx.AsyncClient() as httpx_client:
        resolver = A2ACardResolver(
            httpx_client=httpx_client,
            base_url=base_url,
        )
        card = await resolver.get_agent_card()

        print(f"📇 Agent Card Discovered!\n")
        print(f"   Name:        {card.name}")
        print(f"   Description: {card.description}")
        print(f"   URL:         {card.url}")
        print(f"   Version:     {card.version}")
        print(f"   Streaming:   {card.capabilities.streaming if card.capabilities else 'N/A'}")

        print(f"\n   🎯 Skills ({len(card.skills)}):")
        for skill in card.skills:
            print(f"      • {skill.name} [{skill.id}]")
            print(f"        {skill.description}")
            if skill.examples:
                print(f"        Examples: {skill.examples}")

        return card

currency_card = await discover_agent("http://localhost:5001")


════════════════════════════════════════════════════════════
  Discovering Agent via Agent Card
════════════════════════════════════════════════════════════
📇 Agent Card Discovered!

   Name:        Currency Converter Agent
   Description: Converts between currencies using real-time exchange rates.
   URL:         http://localhost:5001
   Version:     1.0.0
   Streaming:   False

   🎯 Skills (1):
      • Currency Conversion [currency-conversion]
        Convert an amount from one currency to another. Example: '100 USD to JPY'
        Examples: ['100 USD to JPY', '500 EUR to GBP', '1000 INR to USD']


In [56]:
# Cell A12: Send a task to Currency Agent using proper a2a-sdk API

print_header("Sending Task to Currency Agent")

response = await send_task_to_agent(
    "http://localhost:5001",
    "Convert 3000 USD to JPY"
)

# Display the response
print(f"\n📦 Response type: {type(response).__name__}")
print(f"\n📋 Extracted text:")
text = extract_text_from_response(response)
print(f"   {text}")


════════════════════════════════════════════════════════════
  Sending Task to Currency Agent
════════════════════════════════════════════════════════════
  📤 Sending: "Convert 3000 USD to JPY"
  📥 Response received!

📦 Response type: SendMessageResponse

📋 Extracted text:
   3,000.00 USD = 448,500.00 JPY
(Rate: 1 USD = 149.5 JPY)


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


In [57]:
# Cell A12: Test sending a task to Currency Agent

print_header("Sending Task to Currency Agent")

response = await send_task_to_agent(
    "http://localhost:5001",
    "Convert 3000 USD to JPY"
)

# Corrected line: Use .model_dump_json() for Pydantic objects
print(f"\n📦 Raw response:\n{response.model_dump_json(indent=2)[:600]}")

print(f"\n{'─' * 50}")
print(f"📋 Extracted text:")
print(f"   {extract_text_from_response(response)}")


════════════════════════════════════════════════════════════
  Sending Task to Currency Agent
════════════════════════════════════════════════════════════
  📤 Sending: "Convert 3000 USD to JPY"
  📥 Response received!

📦 Raw response:
{
  "id": "5dcb6934-3a53-4dac-8797-9de450e87d94",
  "jsonrpc": "2.0",
  "result": {
    "contextId": null,
    "extensions": null,
    "kind": "message",
    "messageId": "3f03b371-95f0-437a-891c-3786fba57dd9",
    "metadata": null,
    "parts": [
      {
        "kind": "text",
        "metadata": null,
        "text": "3,000.00 USD = 448,500.00 JPY\n(Rate: 1 USD = 149.5 JPY)"
      }
    ],
    "referenceTaskIds": null,
    "role": "agent",
    "taskId": null
  }
}

──────────────────────────────────────────────────
📋 Extracted text:
   3,000.00 USD = 448,500.00 JPY
(Rate: 1 USD = 149.5 JPY)


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


In [58]:
# Cell A13: Parse the A2A response properly

print_header("Parsing A2A Response")

# The response structure depends on the SDK version
# Let's explore what we got
def explore_response(resp, indent=0):
    """Recursively explore a response object."""
    prefix = "  " * indent
    if hasattr(resp, '__dict__'):
        for key, val in resp.__dict__.items():
            if key.startswith('_'):
                continue
            if hasattr(val, '__dict__') and not isinstance(val, str):
                print(f"{prefix}{key}:")
                explore_response(val, indent + 1)
            elif isinstance(val, list):
                print(f"{prefix}{key}: [{len(val)} items]")
                for i, item in enumerate(val[:3]):
                    print(f"{prefix}  [{i}]:")
                    explore_response(item, indent + 2)
            else:
                print(f"{prefix}{key}: {val}")
    else:
        print(f"{prefix}{resp}")

explore_response(response)


════════════════════════════════════════════════════════════
  Parsing A2A Response
════════════════════════════════════════════════════════════
root:
  id: 5dcb6934-3a53-4dac-8797-9de450e87d94
  jsonrpc: 2.0
  result:
    context_id: None
    extensions: None
    kind: message
    message_id: 3f03b371-95f0-437a-891c-3786fba57dd9
    metadata: None
    parts: [1 items]
      [0]:
        root:
          kind: text
          metadata: None
          text: 3,000.00 USD = 448,500.00 JPY
(Rate: 1 USD = 149.5 JPY)
    reference_task_ids: None
    role: Role.agent
    task_id: None


In [19]:
# Cell A13: Parse the A2A response

print_header("Parsing A2A Response")

def explore_response(resp, indent=0, max_depth=5):
    """Recursively explore a response object."""
    if indent > max_depth:
        print(f"{'  ' * indent}... (max depth)")
        return
    prefix = "  " * indent
    if hasattr(resp, '__dict__'):
        for key, val in resp.__dict__.items():
            if key.startswith('_'):
                continue
            if hasattr(val, '__dict__') and not isinstance(val, str):
                print(f"{prefix}{key}:")
                explore_response(val, indent + 1, max_depth)
            elif isinstance(val, list):
                print(f"{prefix}{key}: [{len(val)} items]")
                for i, item in enumerate(val[:3]):
                    print(f"{prefix}  [{i}]:")
                    explore_response(item, indent + 2, max_depth)
            else:
                val_str = str(val)
                if len(val_str) > 100:
                    val_str = val_str[:100] + "..."
                print(f"{prefix}{key}: {val_str}")
    else:
        print(f"{prefix}{resp}")

explore_response(response)


════════════════════════════════════════════════════════════
  Parsing A2A Response
════════════════════════════════════════════════════════════
root:
  id: 7a6d8268-a2a7-47dc-8a70-f0499e2008a5
  jsonrpc: 2.0
  result:
    context_id: None
    extensions: None
    kind: message
    message_id: b0c181b7-72cf-4776-bae3-d080eb7bdd74
    metadata: None
    parts: [1 items]
      [0]:
        root:
          kind: text
          metadata: None
          text: 3,000.00 USD = 448,500.00 JPY
(Rate: 1 USD = 149.5 JPY)
    reference_task_ids: None
    role: Role.agent
    task_id: None


In [59]:
# Cell A14: Test the currency agent with multiple queries

print_header("Testing Currency Agent — Multiple Queries")

test_queries = [
    "100 USD to JPY",
    "500 EUR to GBP",
    "1000 INR to USD",
    "250 USD to EUR",
    "hello what can you do",  # Test help response
]

for query in test_queries:
    print(f"\n💬 Query: \"{query}\"")
    try:
        resp = await send_task_to_agent("http://localhost:5001", query)
        # Try to extract text from response
        if hasattr(resp, 'result'):
            result = resp.result
            if hasattr(result, 'history') and result.history:
                for msg in result.history:
                    text = extract_text_from_message(msg)
                    if text:
                        print(f"   📥 {text}")
            elif hasattr(result, 'artifacts') and result.artifacts:
                for art in result.artifacts:
                    print(f"   📎 Artifact: {art}")
            elif hasattr(result, 'status'):
                print(f"   📊 Status: {result.status}")
        print(f"   {'─' * 40}")
    except Exception as e:
        print(f"   ❌ Error: {e}")


════════════════════════════════════════════════════════════
  Testing Currency Agent — Multiple Queries
════════════════════════════════════════════════════════════

💬 Query: "100 USD to JPY"
  📤 Sending: "100 USD to JPY"
  📥 Response received!
   ────────────────────────────────────────

💬 Query: "500 EUR to GBP"
  📤 Sending: "500 EUR to GBP"
  📥 Response received!
   ────────────────────────────────────────

💬 Query: "1000 INR to USD"
  📤 Sending: "1000 INR to USD"


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  📥 Response received!
   ────────────────────────────────────────

💬 Query: "250 USD to EUR"
  📤 Sending: "250 USD to EUR"
  📥 Response received!
   ────────────────────────────────────────

💬 Query: "hello what can you do"
  📤 Sending: "hello what can you do"
  📥 Response received!
   ────────────────────────────────────────


# 📖 PART 5: A2A Agent with LLM + MCP Inside

**Key Insight:** An A2A agent can be **smart internally** — it can have its own LLM brain and MCP tools.

From the outside, other agents just see the Agent Card. They don't know (or care) what's inside.


In [60]:
%%writefile a2a_agents/finance_agent.py
"""
A2A Agent: Finance Advisor
Uses LLM (OpenAI) internally to analyze budget requests.
"""

import os
import json
import uuid
import uvicorn
from openai import OpenAI

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCard, AgentSkill, AgentCapabilities,
    Message, TextPart, Part,
)


EXCHANGE_RATES = {"USD": 1.0, "JPY": 149.50, "EUR": 0.92, "GBP": 0.79}

COST_OF_LIVING = {
    "Tokyo": {"food": 60, "transport": 15, "activities": 40, "misc": 20},
    "Paris": {"food": 70, "transport": 12, "activities": 50, "misc": 25},
    "London": {"food": 65, "transport": 18, "activities": 45, "misc": 22},
    "New York": {"food": 55, "transport": 15, "activities": 35, "misc": 20},
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_daily_cost_estimate",
            "description": "Get estimated daily costs for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert an amount between currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "from_currency": {"type": "string"},
                    "to_currency": {"type": "string"},
                },
                "required": ["amount", "from_currency", "to_currency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_budget_feasibility",
            "description": "Check if planned expenses fit within a budget",
            "parameters": {
                "type": "object",
                "properties": {
                    "total_budget": {"type": "number"},
                    "flight_cost": {"type": "number"},
                    "hotel_cost": {"type": "number"},
                    "num_days": {"type": "integer"},
                    "city": {"type": "string"},
                },
                "required": ["total_budget"],
            },
        },
    },
]


def execute_tool(name, args):
    if name == "get_daily_cost_estimate":
        city = args.get("city", "").title()
        costs = COST_OF_LIVING.get(city)
        if not costs:
            return json.dumps({"error": f"No data for {city}", "available": list(COST_OF_LIVING.keys())})
        costs_copy = dict(costs)
        costs_copy["total_per_day"] = sum(costs_copy.values())
        return json.dumps({"city": city, "daily_costs_usd": costs_copy})

    elif name == "convert_currency":
        amount = args["amount"]
        from_c = args["from_currency"].upper()
        to_c = args["to_currency"].upper()
        if from_c not in EXCHANGE_RATES or to_c not in EXCHANGE_RATES:
            return json.dumps({"error": "Unknown currency"})
        result = (amount / EXCHANGE_RATES[from_c]) * EXCHANGE_RATES[to_c]
        return json.dumps({"from": from_c, "to": to_c, "amount": amount, "result": round(result, 2)})

    elif name == "analyze_budget_feasibility":
        budget = args["total_budget"]
        flight = args.get("flight_cost", 0)
        hotel = args.get("hotel_cost", 0)
        days = args.get("num_days", 5)
        city = args.get("city", "Tokyo").title()
        costs = COST_OF_LIVING.get(city, {"food": 50, "transport": 15, "activities": 35, "misc": 15})
        daily = sum(costs.values())
        total = flight + hotel + (daily * days)
        remaining = budget - total
        return json.dumps({
            "budget": budget, "total_estimated": round(total, 2),
            "remaining": round(remaining, 2), "feasible": remaining >= 0,
            "breakdown": {"flights": flight, "hotel": hotel, "daily_expenses": daily * days},
            "recommendation": "Within budget!" if remaining >= 0 else f"Over budget by ${abs(remaining):.0f}",
        })

    return json.dumps({"error": f"Unknown tool: {name}"})


def run_llm_with_tools(user_message):
    openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    messages = [
        {
            "role": "system",
            "content": (
                "You are a financial advisor agent specializing in travel budgets. "
                "Use your tools to provide accurate budget analysis. "
                "Always use tools for data. Be concise and structured."
            ),
        },
        {"role": "user", "content": user_message},
    ]

    for _ in range(5):
        response = openai_client.chat.completions.create(
            model="gpt-5.2", messages=messages, tools=TOOLS, temperature=0,
        )
        choice = response.choices[0]

        if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
            messages.append(choice.message.model_dump())
            for tc in choice.message.tool_calls:
                result = execute_tool(tc.function.name, json.loads(tc.function.arguments))
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
        elif choice.finish_reason == "stop":
            return choice.message.content
        else:
            return choice.message.content or "Unable to process request."

    return "Max iterations reached."


class FinanceAgentExecutor(AgentExecutor):

    async def execute(self, context: RequestContext, event_queue: EventQueue):
        user_text = ""
        if context.message and context.message.parts:
            for part in context.message.parts:
                if hasattr(part, "root") and hasattr(part.root, "text"):
                    user_text += part.root.text
                elif hasattr(part, "text"):
                    user_text += part.text

        if not user_text.strip():
            user_text = "What can you help with?"

        result = run_llm_with_tools(user_text)

        # ⭐ THE FIX: await the enqueue_event call!
        await event_queue.enqueue_event(
            Message(
                message_id=str(uuid.uuid4()),
                role="agent",
                parts=[Part(root=TextPart(text=result))],
            )
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue):
        pass


agent_card = AgentCard(
    name="Finance Advisor Agent",
    description="Analyzes travel budgets, estimates costs, converts currencies, and checks financial feasibility using AI.",
    url="http://localhost:5002",
    version="1.0.0",
    defaultInputModes=["text"],
    defaultOutputModes=["text"],
    skills=[
        AgentSkill(
            id="budget-analysis",
            name="Budget Feasibility Analysis",
            description="Analyze if a travel plan fits within a given budget.",
            tags=["finance", "budget", "travel"],
            examples=[
                "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?",
                "Analyze budget: $2000 for 3 days in Paris",
            ],
        ),
        AgentSkill(
            id="cost-estimation",
            name="Daily Cost Estimation",
            description="Estimate daily expenses for a destination city.",
            tags=["finance", "costs"],
            examples=["What are daily costs in Tokyo?"],
        ),
        AgentSkill(
            id="currency-conversion",
            name="Currency Conversion",
            description="Convert amounts between currencies.",
            tags=["finance", "currency"],
            examples=["Convert 3000 USD to JPY"],
        ),
    ],
    capabilities=AgentCapabilities(streaming=False),
)

executor = FinanceAgentExecutor()
task_store = InMemoryTaskStore()
handler = DefaultRequestHandler(agent_executor=executor, task_store=task_store)
app = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)

if __name__ == "__main__":
    print("Finance Advisor Agent starting on port 5002...")
    uvicorn.run(app.build(), host="0.0.0.0", port=5002)

Overwriting a2a_agents/finance_agent.py


In [61]:
# Cell A17: Verify and start the Finance Agent

# Verify
result = subprocess.run(
    [sys.executable, "-c",
     "exec(open('a2a_agents/finance_agent.py').read().split('if __name__')[0])"],
    capture_output=True, text=True, timeout=10,
    env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
)
if result.returncode != 0:
    print(f"❌ Error:\n{result.stderr}")
else:
    print("✅ finance_agent.py verified\n")

# Start
success = start_agent_server("finance-agent", "a2a_agents/finance_agent.py", 5002)

✅ finance_agent.py verified

✅ finance-agent running on port 5002 (PID: 5416)


In [62]:
AUTH_TOKEN = "39e3rgNakBDqcl8i3CGAKMU8Hsc_3drNDoarTnAuURP3mDbXp"

from pyngrok import ngrok
ngrok.set_auth_token(AUTH_TOKEN)

url1 = ngrok.connect(5001)
url2 = ngrok.connect(5002)

print("App1:", url1)
print("App2:", url2)

App1: NgrokTunnel: "https://provocative-splurgily-kelvin.ngrok-free.dev" -> "http://localhost:5001"
App2: NgrokTunnel: "https://provocative-splurgily-kelvin.ngrok-free.dev" -> "http://localhost:5002"


In [23]:
print_step(2, "Expose port 5002 using pyngrok")
from pyngrok import ngrok
import asyncio

# Terminate any existing ngrok tunnels
ngrok.kill()

try:
    # Open a ngrok tunnel to port 5002
    public_url_object = ngrok.connect(5002)
    public_url = public_url_object.public_url

    if public_url:
        print(f"✅ ngrok tunnel established: {public_url}")
        print(f"You can now access your agent at: {public_url}/.well-known/agent.json from your local machine.")
        NGROK_PUBLIC_URL = public_url
    else:
        print("❌ Failed to get ngrok public URL after connection. Check pyngrok logs.")
except Exception as e:
    print(f"❌ An error occurred while trying to connect ngrok: {e}")
    print("Please ensure the ngrok authtoken is correctly set and try again.")



▶ Step 2: Expose port 5002 using pyngrok
───────────────────────────────────────────────────────
✅ ngrok tunnel established: https://provocative-splurgily-kelvin.ngrok-free.dev
You can now access your agent at: https://provocative-splurgily-kelvin.ngrok-free.dev/.well-known/agent.json from your local machine.


**Check the A2A URL by posting in the URL:**

A2A Inspector Project: https://github.com/a2aproject/a2a-inspector

Direct Link: https://a2a-inspector-908687846511.us-central1.run.app/

In [64]:
# Cell A18: Discover the Finance Agent's capabilities

print_header("Finance Agent — Agent Card")
finance_card = await discover_agent("http://localhost:5002")


════════════════════════════════════════════════════════════
  Finance Agent — Agent Card
════════════════════════════════════════════════════════════
📇 Agent Card Discovered!

   Name:        Finance Advisor Agent
   Description: Analyzes travel budgets, estimates costs, converts currencies, and checks financial feasibility using AI.
   URL:         http://localhost:5002
   Version:     1.0.0
   Streaming:   False

   🎯 Skills (3):
      • Budget Feasibility Analysis [budget-analysis]
        Analyze if a travel plan fits within a given budget.
        Examples: ['Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?', 'Analyze budget: $2000 for 3 days in Paris']
      • Daily Cost Estimation [cost-estimation]
        Estimate daily expenses for a destination city.
        Examples: ['What are daily costs in Tokyo?']
      • Currency Conversion [currency-conversion]
        Convert amounts between currencies.
        Examples: ['Convert 3000 USD to JPY']


In [65]:
# Cell A19: Test the Finance Agent — it uses LLM + tools internally!

print_header("Testing Finance Agent (LLM + Tools Inside)")

finance_queries = [
    "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?",
    "What are the estimated daily costs in Tokyo for a moderate traveler?",
    "Convert 3000 USD to JPY",
]

for query in finance_queries:
    print(f"\n💬 Query: \"{query}\"")
    print(f"{'─' * 50}")
    try:
        resp = await send_task_to_agent("http://localhost:5002", query)
        # Extract text from response
        if hasattr(resp, 'result'):
            result = resp.result
            if hasattr(result, 'history') and result.history:
                for msg in result.history:
                    text = extract_text_from_message(msg)
                    if text:
                        print(f"📥 {text}")
        print()
    except Exception as e:
        print(f"❌ Error: {e}\n")


════════════════════════════════════════════════════════════
  Testing Finance Agent (LLM + Tools Inside)
════════════════════════════════════════════════════════════

💬 Query: "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?"
──────────────────────────────────────────────────
  📤 Sending: "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?"


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  📥 Response received!


💬 Query: "What are the estimated daily costs in Tokyo for a moderate traveler?"
──────────────────────────────────────────────────
  📤 Sending: "What are the estimated daily costs in Tokyo for a moderate traveler?"
  📥 Response received!


💬 Query: "Convert 3000 USD to JPY"
──────────────────────────────────────────────────
  📤 Sending: "Convert 3000 USD to JPY"
  📥 Response received!



In [66]:
print(resp)

root=SendMessageSuccessResponse(id='3481c7f1-1e81-4d52-92c3-b93a64259dac', jsonrpc='2.0', result=Message(context_id=None, extensions=None, kind='message', message_id='a235f547-d1d4-4788-81de-c33a26344f53', metadata=None, parts=[Part(root=TextPart(kind='text', metadata=None, text='3,000 USD ≈ **448,500 JPY**.'))], reference_task_ids=None, role=<Role.agent: 'agent'>, task_id=None))


In [67]:
# Cell A20: Compare the two agents

print_header("Comparison: Simple Agent vs LLM-Powered Agent")

print("""
┌──────────────────────┬───────────────────────┬────────────────────────┐
│                      │ Currency Agent         │ Finance Agent          │
│                      │ (Port 5001)            │ (Port 5002)            │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ Internal Brain       │ ❌ No LLM              │ ✅ GPT-4o-mini         │
│ Internal Tools       │ Simple regex + math    │ 3 tools via LLM       │
│ Response Style       │ Fixed format           │ Natural language       │
│ Flexibility          │ Rigid parsing          │ Understands context    │
│ Speed                │ ⚡ Very fast            │ 🐢 Slower (LLM call)  │
│ Cost                 │ 💰 Free                │ 💸 API costs           │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ From outside (A2A)   │ Looks the same!        │ Looks the same!        │
│ Agent Card           │ ✅ Standard             │ ✅ Standard             │
│ Message format       │ ✅ Same                 │ ✅ Same                 │
└──────────────────────┴───────────────────────┴────────────────────────┘

💡 KEY INSIGHT: A2A doesn't care what's INSIDE the agent!
   Simple logic, LLM, MCP tools, database — all look the same from outside.
""")


════════════════════════════════════════════════════════════
  Comparison: Simple Agent vs LLM-Powered Agent
════════════════════════════════════════════════════════════

┌──────────────────────┬───────────────────────┬────────────────────────┐
│                      │ Currency Agent         │ Finance Agent          │
│                      │ (Port 5001)            │ (Port 5002)            │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ Internal Brain       │ ❌ No LLM              │ ✅ GPT-4o-mini         │
│ Internal Tools       │ Simple regex + math    │ 3 tools via LLM       │
│ Response Style       │ Fixed format           │ Natural language       │
│ Flexibility          │ Rigid parsing          │ Understands context    │
│ Speed                │ ⚡ Very fast            │ 🐢 Slower (LLM call)  │
│ Cost                 │ 💰 Free                │ 💸 API costs           │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ From outsid

# 📖 PART 6: Multi-Agent Discovery & LLM Delegation

**Goal:** Show how an LLM orchestrator can:
1. **Discover** multiple A2A agents
2. **Decide** which agent to delegate to
3. **Send tasks** and collect results

                ┌──────────────────────┐
                │   LLM Orchestrator   │
                │                      │
                │  "Which agent should  │
                │   handle this task?"  │
                └───┬─────────┬────────┘
                    │         │
               A2A  │    A2A  │
                    ▼         ▼
          ┌──────────┐  ┌──────────┐
          │ Currency │  │ Finance  │
          │ Agent    │  │ Agent    │
          │ :5001    │  │ :5002    │
          └──────────┘  └──────────┘

In [68]:
# Cell A22: Discover all running A2A agents

print_header("Discovering All Available A2A Agents")

AGENT_URLS = {
    "currency-agent": "http://localhost:5001",
    "finance-agent": "http://localhost:5002",
}

discovered_agents = {}

for name, url in AGENT_URLS.items():
    try:
        card = await discover_agent(url)
        discovered_agents[name] = {"card": card, "url": url}
        print()
    except Exception as e:
        print(f"  ❌ {name} at {url}: {e}\n")

print(f"\n{'═' * 55}")
print(f"  📋 Discovered {len(discovered_agents)} agent(s)")
print(f"{'═' * 55}")


════════════════════════════════════════════════════════════
  Discovering All Available A2A Agents
════════════════════════════════════════════════════════════
📇 Agent Card Discovered!

   Name:        Currency Converter Agent
   Description: Converts between currencies using real-time exchange rates.
   URL:         http://localhost:5001
   Version:     1.0.0
   Streaming:   False

   🎯 Skills (1):
      • Currency Conversion [currency-conversion]
        Convert an amount from one currency to another. Example: '100 USD to JPY'
        Examples: ['100 USD to JPY', '500 EUR to GBP', '1000 INR to USD']

📇 Agent Card Discovered!

   Name:        Finance Advisor Agent
   Description: Analyzes travel budgets, estimates costs, converts currencies, and checks financial feasibility using AI.
   URL:         http://localhost:5002
   Version:     1.0.0
   Streaming:   False

   🎯 Skills (3):
      • Budget Feasibility Analysis [budget-analysis]
        Analyze if a travel plan fits within a g

In [69]:
# Cell A23: Build a description of all agents for the LLM to understand

def build_agent_registry_prompt(agents_dict):
    """Create a text description of all agents for the LLM."""
    lines = ["You have access to the following A2A agents:\n"]

    for name, info in agents_dict.items():
        card = info["card"]
        lines.append(f"AGENT: {card.name}")
        lines.append(f"  ID: {name}")
        lines.append(f"  URL: {info['url']}")
        lines.append(f"  Description: {card.description}")
        lines.append(f"  Skills:")
        for skill in card.skills:
            lines.append(f"    - {skill.name}: {skill.description}")
            if skill.examples:
                lines.append(f"      Examples: {', '.join(skill.examples[:2])}")
        lines.append("")

    return "\n".join(lines)

registry_prompt = build_agent_registry_prompt(discovered_agents)
print("📋 Agent Registry (what the LLM sees):\n")
print(registry_prompt)

📋 Agent Registry (what the LLM sees):

You have access to the following A2A agents:

AGENT: Currency Converter Agent
  ID: currency-agent
  URL: http://localhost:5001
  Description: Converts between currencies using real-time exchange rates.
  Skills:
    - Currency Conversion: Convert an amount from one currency to another. Example: '100 USD to JPY'
      Examples: 100 USD to JPY, 500 EUR to GBP

AGENT: Finance Advisor Agent
  ID: finance-agent
  URL: http://localhost:5002
  Description: Analyzes travel budgets, estimates costs, converts currencies, and checks financial feasibility using AI.
  Skills:
    - Budget Feasibility Analysis: Analyze if a travel plan fits within a given budget.
      Examples: Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?, Analyze budget: $2000 for 3 days in Paris
    - Daily Cost Estimation: Estimate daily expenses for a destination city.
      Examples: What are daily costs in Tokyo?
    - Currency Conversion: Convert amounts b

In [70]:
# Cell A24: LLM as orchestrator — decides which agent to delegate to

print_header("LLM Orchestrator — Agent Selection")

ORCHESTRATOR_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "delegate_to_agent",
            "description": "Send a task to an A2A agent for processing",
            "parameters": {
                "type": "object",
                "properties": {
                    "agent_id": {
                        "type": "string",
                        "description": "ID of the agent to delegate to",
                        "enum": list(discovered_agents.keys()),
                    },
                    "message": {
                        "type": "string",
                        "description": "The task/message to send to the agent",
                    },
                },
                "required": ["agent_id", "message"],
            },
        },
    },
]

async def llm_orchestrator(user_query):
    """LLM decides which agent to use and delegates."""
    print(f"💬 User: {user_query}\n")

    messages = [
        {
            "role": "system",
            "content": f"""You are an orchestrator that delegates tasks to specialized A2A agents.
Analyze the user's request and decide which agent(s) to delegate to.
Use the delegate_to_agent tool to send tasks.

{registry_prompt}

Rules:
- Use currency-agent for simple currency conversions
- Use finance-agent for budget analysis, cost estimation, or financial advice
- You can call multiple agents if needed
- After getting results, compile a final response for the user"""
        },
        {"role": "user", "content": user_query},
    ]

    all_agent_results = []

    for iteration in range(5):
        response = client_openai.chat.completions.create(
            model=MODEL, messages=messages, tools=ORCHESTRATOR_TOOLS, temperature=0,
        )
        choice = response.choices[0]

        if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
            messages.append(choice.message.model_dump())

            for tc in choice.message.tool_calls:
                args = json.loads(tc.function.arguments)
                agent_id = args["agent_id"]
                agent_message = args["message"]

                print(f"  🤖 LLM delegates to: {agent_id}")
                print(f"     Message: \"{agent_message}\"")

                # Actually call the A2A agent!
                agent_url = discovered_agents[agent_id]["url"]
                try:
                    resp = await send_task_to_agent(agent_url, agent_message)

                    # Extract response text
                    result_text = ""
                    if hasattr(resp, 'result') and hasattr(resp.result, 'history'):
                        for msg in resp.result.history:
                            text = extract_text_from_message(msg)
                            if text:
                                result_text += text + "\n"

                    if not result_text:
                        result_text = str(resp)

                    print(f"  📥 Agent response: {result_text[:150]}...")
                    all_agent_results.append({"agent": agent_id, "result": result_text})

                except Exception as e:
                    result_text = f"Error calling agent: {e}"
                    print(f"  ❌ {result_text}")

                messages.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": result_text,
                })

        elif choice.finish_reason == "stop":
            final = choice.message.content
            print(f"\n{'═' * 50}")
            print(f"📋 Final Orchestrated Response:\n")
            print(final)
            return final, all_agent_results

    return "Max iterations reached", all_agent_results


# Test 1: Simple delegation
result, agents_used = await llm_orchestrator("How much is 2000 USD in Japanese Yen?")


════════════════════════════════════════════════════════════
  LLM Orchestrator — Agent Selection
════════════════════════════════════════════════════════════
💬 User: How much is 2000 USD in Japanese Yen?

  🤖 LLM delegates to: currency-agent
     Message: "Convert 2000 USD to JPY using real-time exchange rate."
  📤 Sending: "Convert 2000 USD to JPY using real-time exchange rate."
  📥 Response received!
  📥 Agent response: root=SendMessageSuccessResponse(id='a09b47d4-76de-444c-81f2-2c0d851ee74a', jsonrpc='2.0', result=Message(context_id=None, extensions=None, kind='messa...


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)



══════════════════════════════════════════════════
📋 Final Orchestrated Response:

2,000 USD ≈ **299,000 JPY** (using **1 USD = 149.5 JPY**).


In [71]:
# Cell A25: Test — LLM should choose the Finance Agent

print("\n\n")
result, agents_used = await llm_orchestrator(
    "Is a 5-day trip to Tokyo feasible with a $3000 budget? "
    "Flights cost $820 and hotel costs $950."
)




💬 User: Is a 5-day trip to Tokyo feasible with a $3000 budget? Flights cost $820 and hotel costs $950.

  🤖 LLM delegates to: finance-agent
     Message: "Analyze if a 5-day Tokyo trip is feasible with $3000 budget given flights $820 and hotel $950. Include remaining budget, estimated daily costs, and conclusion."
  📤 Sending: "Analyze if a 5-day Tokyo trip is feasible with $3000 budget given flights $820 and hotel $950. Include remaining budget, estimated daily costs, and conclusion."


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  📥 Response received!
  📥 Agent response: root=SendMessageSuccessResponse(id='9193681d-0e04-451b-9c4b-60e46d2de21c', jsonrpc='2.0', result=Message(context_id=None, extensions=None, kind='messa...

══════════════════════════════════════════════════
📋 Final Orchestrated Response:

Yes—based on typical Tokyo costs, it looks feasible.

- **Budget:** $3,000  
- **Flights:** $820  
- **Hotel:** $950  
- **Fixed costs total:** $1,770  
- **Money left after flights + hotel:** $3,000 − $1,770 = **$1,230**

**Estimated daily spending in Tokyo (mid-range):** about **$135/day** (food ~$60, transport ~$15, activities ~$40, misc ~$20)  
- For **5 days:** $135 × 5 = **$675**

**Estimated trip total:** $1,770 + $675 = **$2,445**  
**Remaining buffer:** $3,000 − $2,445 = **$555**

**Conclusion:** Feasible, with roughly **$555** left for extras (shopping, day trips, nicer meals) or unexpected costs.


In [72]:
# Cell A26: Test — LLM should use BOTH agents

print("\n\n")
result, agents_used = await llm_orchestrator(
    "I have a budget of 450000 JPY for a Tokyo trip. "
    "First convert that to USD, then tell me if a 5-day trip "
    "with $820 flights and $950 hotel is feasible within that budget."
)

print(f"\n📊 Agents used: {[a['agent'] for a in agents_used]}")




💬 User: I have a budget of 450000 JPY for a Tokyo trip. First convert that to USD, then tell me if a 5-day trip with $820 flights and $950 hotel is feasible within that budget.

  🤖 LLM delegates to: currency-agent
     Message: "Convert 450000 JPY to USD."
  📤 Sending: "Convert 450000 JPY to USD."
  📥 Response received!
  📥 Agent response: root=SendMessageSuccessResponse(id='1b0154fc-2c01-4f1e-ac64-535f388097b0', jsonrpc='2.0', result=Message(context_id=None, extensions=None, kind='messa...
  🤖 LLM delegates to: finance-agent
     Message: "Budget feasibility: 5-day Tokyo trip. Budget is 450000 JPY (also provide in USD if possible). Known costs: $820 flights, $950 hotel. Determine feasibility and remaining budget estimate."
  📤 Sending: "Budget feasibility: 5-day Tokyo trip. Budget is 450000 JPY (also provide in USD if possible). Known costs: $820 flights, $950 hotel. Determine feasibility and remaining budget estimate."


/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  📥 Response received!
  📥 Agent response: root=SendMessageSuccessResponse(id='bf36a022-bcba-4d92-b40d-8b505b46e86b', jsonrpc='2.0', result=Message(context_id=None, extensions=None, kind='messa...

══════════════════════════════════════════════════
📋 Final Orchestrated Response:

**1) Convert 450,000 JPY to USD**  
- **450,000 JPY ≈ $3,010.03 USD** (at ~1 JPY = $0.0067)

**2) Feasibility of a 5-day Tokyo trip with $820 flights + $950 hotel**  
- Fixed costs:  
  - Flights: **$820**  
  - Hotel: **$950**  
  - **Fixed subtotal: $1,770**

- Remaining from budget:  
  - **$3,010.03 − $1,770 = $1,240.03** left for 5 days

- Using a reasonable mid-range daily spend estimate for Tokyo (~**$135/day**):  
  - 5 days ≈ **$675**

**Result:** **Yes, it’s feasible within your budget.**  
- Estimated total: **$1,770 + $675 = $2,445**  
- Estimated cushion: **$3,010.03 − $2,445 ≈ $565** (≈ **84,473 JPY**)

📊 Agents used: ['currency-agent', 'finance-agent']


# 📖 PART 7: Grand Finale — LLM + MCP Tools + A2A Agents

The **complete agentic system** where:

- 🧠 **LLM Orchestrator** is the central brain
- 🔧 **MCP Tools** give it direct capabilities (flights, hotels, calendar)
- 🤖 **A2A Agents** handle specialized delegated tasks (finance, currency)
- The LLM **decides** when to use its own tools vs delegate to agents

                     ┌─────────────┐
                     │    USER     │
                     └──────┬──────┘
                            │
                ┌───────────▼────────────┐
                │  🧠 LLM ORCHESTRATOR   │
                │                        │
                │  Decides:              │
                │  • Use MCP tool?       │
                │  • Delegate to agent?  │
                └──┬──────────┬──────────┘
                   │          │
          MCP Tools│          │ A2A Protocol
      ┌────────────┘          └──────────────┐
      │                                       │
┌─────▼──────┐                    ┌───────────▼────────┐
│ MCP Server │                    │   A2A Agents       │
│            │                    │                    │
│ • flights  │                    │ • Currency :5001   │
│ • hotels   │                    │ • Finance  :5002   │
│ • calendar │                    │   (LLM+tools)     │
│ • expenses │                    │                    │
└────────────┘                    └────────────────────┘

In [33]:
# Cell 18: We'll write our MCP server to a file
# The server runs as a SEPARATE PROCESS — spawned via stdio
# Let's first create the directory

import os
os.makedirs("mcp_server", exist_ok=True)
print("✅ Created mcp_server/ directory")

✅ Created mcp_server/ directory


In [34]:
%%writefile mcp_server/travel_server.py
"""
Travel Planning MCP Server — Built with FastMCP 2.x
"""

import json
import sys
from datetime import datetime

try:
    from fastmcp import FastMCP
except ImportError as e:
    print(f"Import error: {e}", file=sys.stderr)
    sys.exit(1)

# ═══════════════════════════════════════════
# Create the MCP Server (FastMCP 2.x API)
# ═══════════════════════════════════════════
mcp = FastMCP("Travel Planning Tools")

# ═══════════════════════════════════════════
# SIMULATED DATA
# ═══════════════════════════════════════════

FLIGHT_DATABASE = {
    ("SFO", "TYO"): [
        {"airline": "ANA", "flight": "NH7", "depart": "11:30 AM", "arrive": "3:30 PM +1",
         "duration": "11h 00m", "price": 820, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL1", "depart": "1:00 PM", "arrive": "5:15 PM +1",
         "duration": "11h 15m", "price": 910, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA837", "depart": "2:45 PM", "arrive": "7:00 PM +1",
         "duration": "11h 15m", "price": 750, "stops": 0, "window_seat": False},
        {"airline": "Delta (via Seoul)", "flight": "DL295", "depart": "9:00 AM", "arrive": "8:30 PM +1",
         "duration": "17h 30m", "price": 620, "stops": 1, "window_seat": True},
    ],
    ("TYO", "SFO"): [
        {"airline": "ANA", "flight": "NH8", "depart": "5:00 PM", "arrive": "10:30 AM same day",
         "duration": "9h 30m", "price": 840, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL2", "depart": "6:30 PM", "arrive": "11:45 AM same day",
         "duration": "9h 15m", "price": 890, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA838", "depart": "4:00 PM", "arrive": "9:30 AM same day",
         "duration": "9h 30m", "price": 770, "stops": 0, "window_seat": True},
    ],
}

HOTEL_DATABASE = {
    "Tokyo": [
        {"name": "K5 Tokyo", "area": "Nihonbashi", "type": "boutique",
         "rating": 4.8, "price_night": 190, "vegetarian": True},
        {"name": "Trunk Hotel", "area": "Shibuya", "type": "boutique",
         "rating": 4.6, "price_night": 220, "vegetarian": True},
        {"name": "MUJI Hotel Ginza", "area": "Ginza", "type": "boutique",
         "rating": 4.5, "price_night": 175, "vegetarian": True},
        {"name": "APA Hotel Shinjuku", "area": "Shinjuku", "type": "business",
         "rating": 3.8, "price_night": 95, "vegetarian": False},
        {"name": "Park Hyatt Tokyo", "area": "Shinjuku", "type": "luxury",
         "rating": 4.9, "price_night": 550, "vegetarian": True},
    ]
}

CALENDAR = [
    {"date": "2026-10-01", "time": "09:00-10:00", "event": "Team standup", "flexible": True},
    {"date": "2026-10-03", "time": "14:00-15:00", "event": "Dentist appointment", "flexible": False},
    {"date": "2026-10-07", "time": "11:00-12:00", "event": "Project review", "flexible": False},
]

USER_PREFS = {
    "name": "Alex Johnson", "home_airport": "SFO",
    "seat": "window", "meal": "vegetarian", "hotel_type": "boutique",
    "budget_style": "moderate",
    "loyalty": ["ANA Mileage Club", "Marriott Bonvoy"],
    "interests": ["Japanese cuisine", "temples", "street photography", "local markets"],
    "past_trips": ["Kyoto 2023", "Osaka 2024"],
}

TOKYO_TIPS = {
    "best_months": ["March-April (Cherry Blossom)", "October-November (Autumn)"],
    "must_visit": ["Senso-ji Temple", "Meiji Shrine", "Tsukiji Market", "Shibuya Crossing"],
    "vegetarian_spots": ["Ain Soph Ripple (Shinjuku)", "T's TanTan (Tokyo Station)", "Bon (Taito)"],
    "tip": "Get a 72-hour Tokyo Metro Pass for 1,500 yen (~$10)",
}


# ═══════════════════════════════════════════
# TOOLS
# ═══════════════════════════════════════════

@mcp.tool()
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search flights between two airports on a specific date. Returns real pricing and availability."""
    origin = origin.upper().strip()
    destination = destination.upper().strip()
    code_map = {"TOKYO": "TYO", "NRT": "TYO", "HND": "TYO", "SAN FRANCISCO": "SFO"}
    origin = code_map.get(origin, origin)
    destination = code_map.get(destination, destination)

    flights = FLIGHT_DATABASE.get((origin, destination), [])
    results = [dict(f, date=date) for f in flights]

    return json.dumps({
        "route": f"{origin} -> {destination}", "date": date,
        "count": len(results), "flights": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def search_hotels(city: str, checkin: str, checkout: str, hotel_type: str = "any") -> str:
    """Search hotels in a city with pricing. Filter by type: boutique, business, luxury, or any."""
    hotels = HOTEL_DATABASE.get(city.strip().title(), [])

    if hotel_type != "any":
        hotels = [h for h in hotels if h["type"] == hotel_type.lower()]

    try:
        nights = (datetime.strptime(checkout, "%Y-%m-%d") - datetime.strptime(checkin, "%Y-%m-%d")).days
    except ValueError:
        nights = 5

    results = [dict(h, checkin=checkin, checkout=checkout, nights=nights,
                    total_price=h["price_night"] * nights) for h in hotels]
    results.sort(key=lambda x: x["rating"], reverse=True)

    return json.dumps({
        "city": city.title(), "checkin": checkin, "checkout": checkout,
        "nights": nights, "count": len(results), "hotels": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def check_calendar(start_date: str, end_date: str) -> str:
    """Check user calendar for events and conflicts in a date range."""
    try:
        d1 = datetime.strptime(start_date, "%Y-%m-%d")
        d2 = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return json.dumps({"error": "Invalid date format. Use YYYY-MM-DD"})

    events_in_range = []
    conflicts = []
    for ev in CALENDAR:
        ev_date = datetime.strptime(ev["date"], "%Y-%m-%d")
        if d1 <= ev_date <= d2:
            events_in_range.append(ev)
            if not ev["flexible"]:
                conflicts.append(ev)

    return json.dumps({
        "range": f"{start_date} to {end_date}",
        "events": events_in_range, "conflicts": conflicts,
        "conflict_count": len(conflicts),
    }, indent=2)


@mcp.tool()
def estimate_daily_expenses(city: str, style: str = "moderate") -> str:
    """Estimate daily expenses for a city. Style: budget, moderate, or luxury."""
    costs = {
        "Tokyo": {
            "budget": {"food": 30, "transport": 10, "activities": 20, "misc": 10},
            "moderate": {"food": 60, "transport": 15, "activities": 40, "misc": 20},
            "luxury": {"food": 150, "transport": 50, "activities": 100, "misc": 50},
        }
    }
    city_costs = costs.get(city.title(), {}).get(style.lower())
    if not city_costs:
        return json.dumps({"error": f"No data for {city}/{style}"})

    city_costs["total_per_day"] = sum(city_costs.values())
    return json.dumps({"city": city.title(), "style": style, "daily_usd": city_costs}, indent=2)


@mcp.tool()
def calculate_trip_budget(
    flight_cost: float, hotel_cost: float, daily_expense: float,
    num_days: int, total_budget: float
) -> str:
    """Calculate total trip cost and check if it fits within budget."""
    daily_total = daily_expense * num_days
    total = flight_cost + hotel_cost + daily_total
    remaining = total_budget - total

    return json.dumps({
        "breakdown": {"flights": flight_cost, "hotel": hotel_cost,
                      "daily_expenses": daily_total, "total": total},
        "budget": total_budget, "remaining": remaining,
        "feasible": remaining >= 0,
        "verdict": (f"Within budget! ${remaining:.0f} left." if remaining >= 0
                    else f"Over by ${abs(remaining):.0f}.")
    }, indent=2)


# ═══════════════════════════════════════════
# RESOURCES
# ═══════════════════════════════════════════

@mcp.resource("user://travel-preferences")
def get_user_preferences() -> str:
    """User travel preferences and profile."""
    return json.dumps(USER_PREFS, indent=2)


@mcp.resource("data://tokyo-travel-tips")
def get_tokyo_tips() -> str:
    """Local travel tips for Tokyo."""
    return json.dumps(TOKYO_TIPS, indent=2)


# ═══════════════════════════════════════════
# PROMPTS
# ═══════════════════════════════════════════

@mcp.prompt()
def trip_planner_prompt(destination: str, days: int, budget: float) -> str:
    """System prompt for comprehensive trip planning."""
    return f"""You are an expert travel planner. Plan a {days}-day trip to {destination} within ${budget}.

Steps:
1. Read user://travel-preferences for personalization
2. Use check_calendar to find scheduling conflicts
3. Use search_flights for real flight options
4. Use search_hotels matching user hotel type preference
5. Use estimate_daily_expenses for cost estimation
6. Use calculate_trip_budget to verify feasibility
7. Compile a comprehensive, personalized plan

IMPORTANT: Use REAL data from tools. Never make up prices."""


# ═══════════════════════════════════════════
# Run
# ═══════════════════════════════════════════
if __name__ == "__main__":
    mcp.run(transport="stdio")

Writing mcp_server/travel_server.py


In [73]:
# Cell 20: Verify the server file is valid and runs without errors

import subprocess, sys, os

server_path = os.path.abspath("mcp_server/travel_server.py")

print(f"📄 Server: {server_path}")
print(f"📂 Exists: {os.path.exists(server_path)}\n")

# Test: can it start without crashing?
result = subprocess.run(
    [sys.executable, server_path],
    capture_output=True, text=True, timeout=5, input=""
)

if result.returncode == 0 or "Traceback" not in result.stderr:
    print("✅ Server starts cleanly! No import/syntax errors.")

    # Count what we defined
    with open(server_path) as f:
        content = f.read()
    tools = content.count("@mcp.tool()")
    resources = content.count("@mcp.resource(")
    prompts = content.count("@mcp.prompt()")
    print(f"\n📦 Server contents:")
    print(f"   🔧 Tools:     {tools}")
    print(f"   📄 Resources: {resources}")
    print(f"   📝 Prompts:   {prompts}")
else:
    print(f"❌ Server has errors:\n")
    print(result.stderr)

📄 Server: /content/mcp_server/travel_server.py
📂 Exists: True

✅ Server starts cleanly! No import/syntax errors.

📦 Server contents:
   🔧 Tools:     5
   📄 Resources: 2
   📝 Prompts:   1


In [74]:
# Cell A28: Define all tools — MCP tools + A2A delegation

# MCP tools (from Part 3) — converted to OpenAI format
# We'll call them directly via MCP

# A2A delegation tool
DELEGATION_TOOL = {
    "type": "function",
    "function": {
        "name": "delegate_to_a2a_agent",
        "description": (
            "Delegate a task to a specialized A2A agent. "
            "Use this for financial analysis, budget checks, and currency conversion. "
            "Available agents: "
            "currency-agent (simple currency conversion), "
            "finance-agent (budget analysis, cost estimation, financial advice using AI)"
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "agent_id": {
                    "type": "string",
                    "enum": ["currency-agent", "finance-agent"],
                    "description": "Which agent to delegate to",
                },
                "message": {
                    "type": "string",
                    "description": "Task description for the agent",
                },
            },
            "required": ["agent_id", "message"],
        },
    },
}

print("✅ Delegation tool defined")
print(f"   Available agents: {DELEGATION_TOOL['function']['parameters']['properties']['agent_id']['enum']}")

✅ Delegation tool defined
   Available agents: ['currency-agent', 'finance-agent']


In [75]:
# Cell A29: The Grand Orchestrator — MCP tools + A2A agents + LLM brain

from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession

MCP_SERVER_SCRIPT = os.path.abspath("mcp_server/travel_server.py")

async def grand_orchestrator(user_query):
    """
    The complete agentic system:
    - LLM brain decides what to do
    - MCP tools for direct capabilities (flights, hotels, calendar)
    - A2A agents for delegated tasks (finance, currency)
    """
    print_header("GRAND ORCHESTRATOR: LLM + MCP + A2A")
    print(f"\n💬 User: {user_query}\n")

    # ── Step 1: Connect to MCP server ──
    print_step(1, "Connecting to MCP server for tools")
    server_params = StdioServerParameters(
        command=sys.executable,
        args=[MCP_SERVER_SCRIPT],
        env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
    )

    # FIX: Provide a writable file object for errlog to avoid UnsupportedOperation: fileno error in Colab
    # Redirect stderr of the child process to /dev/null for the mcp stdio_client
    with open(os.devnull, 'w') as devnull_file:
        async with stdio_client(server_params, errlog=devnull_file) as (read, write):
            async with ClientSession(read, write) as mcp_session:
                await mcp_session.initialize()
                print("  ✅ MCP server connected")

                # ── Step 2: Get MCP tools ──
                tools_result = await mcp_session.list_tools()
                mcp_tools = tools_result.tools
                print(f"  🔧 MCP tools: {[t.name for t in mcp_tools]}")

                # Convert MCP tools to OpenAI format
                openai_mcp_tools = []
                for tool in mcp_tools:
                    openai_mcp_tools.append({
                        "type": "function",
                        "function": {
                            "name": tool.name,
                            "description": tool.description or "",
                            "parameters": tool.inputSchema if tool.inputSchema else {
                                "type": "object", "properties": {}
                            },
                        },
                    })

                # ── Step 3: Add A2A delegation tool ──
                a2a_delegation_tool = {
                    "type": "function",
                    "function": {
                        "name": "delegate_to_a2a_agent",
                        "description": (
                            "Delegate a task to a specialized A2A agent. "
                            "Use 'currency-agent' for simple currency conversions. "
                            "Use 'finance-agent' for budget analysis, cost feasibility, "
                            "and financial advice (this agent uses AI internally)."
                        ),
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "agent_id": {
                                    "type": "string",
                                    "enum": ["currency-agent", "finance-agent"],
                                    "description": "Which agent to delegate to",
                                },
                                "message": {
                                    "type": "string",
                                    "description": "The task/question for the agent",
                                },
                            },
                            "required": ["agent_id", "message"],
                        },
                    },
                }

                # Combine all tools
                all_tools = openai_mcp_tools + [a2a_delegation_tool]
                print(f"  🤖 A2A agents: currency-agent, finance-agent")
                print(f"  📊 Total tools available to LLM: {len(all_tools)}")

                # ── Step 4: Read MCP resources for context ──
                print_step(2, "Reading MCP resources for context")
                prefs = await mcp_session.read_resource("user://travel-preferences")
                prefs_text = prefs.contents[0].text if prefs.contents else "{}"

                tips = await mcp_session.read_resource("data://tokyo-travel-tips")
                tips_text = tips.contents[0].text if tips.contents else "{}"
                print("  📄 Loaded: user preferences + Tokyo tips")

                # ── Step 5: Build system prompt ──
                system_message = f"""You are an intelligent travel planning orchestrator.

Youhave TWO types of capabilities:

## 1. MCP Tools (direct — use for search and data):
- search_flights: Search real flights between airports
- search_hotels: Search hotels with pricing
- check_calendar: Check user's calendar for conflicts
- estimate_daily_expenses: Get daily cost estimates for a city
- calculate_trip_budget: Calculate total budget and feasibility

## 2. A2A Agents (delegate — use for specialized analysis):
- currency-agent: Fast currency conversion (e.g., "100 USD to JPY")
- finance-agent: Intelligent budget analysis using AI (complex financial questions)

## Decision Rules:
- Use MCP tools for: flight search, hotel search, calendar checks
- Use currency-agent for: simple currency conversions
- Use finance-agent for: complex budget analysis, financial recommendations
- You CAN combine MCP tools AND A2A agents in the same workflow
- After gathering all data, compile a comprehensive response

## User Preferences:
{prefs_text}

## Destination Tips:
{tips_text}
"""

                messages = [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_query},
                ]

                # ── Step 6: Agentic Loop ──
                print_step(3, "Starting agentic loop (LLM decides tools vs agents)")

                stats = {
                    "iterations": 0,
                    "mcp_tool_calls": 0,
                    "a2a_delegations": 0,
                    "mcp_tools_used": set(),
                    "agents_used": set(),
                }

                for iteration in range(1, 12):
                    stats["iterations"] = iteration
                    print(f"\n  🔄 Iteration {iteration}")
                    print(f"  {'─' * 45}")

                    response = client_openai.chat.completions.create(
                        model=MODEL,
                        messages=messages,
                        tools=all_tools,
                        temperature=0,
                    )
                    choice = response.choices[0]

                    # ─── LLM wants to call tool(s) ───
                    if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
                        messages.append(choice.message.model_dump())

                        for tc in choice.message.tool_calls:
                            func_name = tc.function.name
                            func_args = json.loads(tc.function.arguments)

                            # ═══ CASE A: A2A Agent Delegation ═══
                            if func_name == "delegate_to_a2a_agent":
                                agent_id = func_args["agent_id"]
                                agent_msg = func_args["message"]
                                stats["a2a_delegations"] += 1
                                stats["agents_used"].add(agent_id)

                                print(f"  🤖 A2A DELEGATION → {agent_id}")
                                print(f"     Message: \"{agent_msg[:80]}...\"" if len(agent_msg) > 80 else f"     Message: \"{agent_msg}\"")

                                agent_url = discovered_agents[agent_id]["url"]
                                try:
                                    resp = await send_task_to_agent(agent_url, agent_msg)

                                    result_text = ""
                                    if hasattr(resp, 'result') and hasattr(resp.result, 'history'):
                                        for msg in resp.result.history:
                                            text = extract_text_from_message(msg)
                                            if text:
                                                result_text += text + "\n"
                                    if not result_text:
                                        result_text = str(resp)

                                    print(f"  📥 Agent response ({len(result_text)} chars)")

                                except Exception as e:
                                    result_text = f"Agent error: {e}"
                                    print(f"  ❌ {result_text}")

                                messages.append({
                                    "role": "tool",
                                    "tool_call_id": tc.id,
                                    "content": result_text,
                                })

                            # ═══ CASE B: MCP Tool Call ═══
                            else:
                                stats["mcp_tool_calls"] += 1
                                stats["mcp_tools_used"].add(func_name)

                                print(f"  🔧 MCP TOOL → {func_name}")
                                print(f"     Args: {json.dumps(func_args)[:80]}")

                                try:
                                    mcp_result = await mcp_session.call_tool(func_name, func_args)
                                    result_text = ""
                                    for block in mcp_result.content:
                                        if hasattr(block, "text"):
                                            result_text += block.text
                                    print(f"  📥 Tool result ({len(result_text)} chars)")

                                except Exception as e:
                                    result_text = json.dumps({"error": str(e)})
                                    print(f"  ❌ MCP error: {e}")

                                messages.append({
                                    "role": "tool",
                                    "tool_call_id": tc.id,
                                    "content": result_text,
                                })

                    # ─── LLM is done ───
                    elif choice.finish_reason == "stop":
                        print(f"\n  ✅ LLM finished!")
                        final_answer = choice.message.content or "No response"
                        return final_answer, stats

                    else:
                        if choice.message.content:
                            return choice.message.content, stats
                        messages.append(choice.message.model_dump())

                return "Max iterations reached", stats


print("✅ Grand orchestrator function defined!")
print("   Combines: MCP tools + A2A agents + LLM brain")

✅ Grand orchestrator function defined!
   Combines: MCP tools + A2A agents + LLM brain


In [76]:
# Cell A30: Test 1 — Simple flight search (should use MCP tools)

print("=" * 60)
print("  TEST 1: Simple flight search → expects MCP tools")
print("=" * 60)

answer, stats = await grand_orchestrator(
    "Search for flights from SFO to Tokyo on 2026-10-01"
)

print(f"\n{'═' * 55}")
print("📋 FINAL ANSWER:")
print(f"{'═' * 55}")
print(answer)

print(f"\n📊 Stats: MCP calls={stats['mcp_tool_calls']}, A2A delegations={stats['a2a_delegations']}")
print(f"   MCP tools: {stats['mcp_tools_used']}")
print(f"   A2A agents: {stats['agents_used']}")

  TEST 1: Simple flight search → expects MCP tools

════════════════════════════════════════════════════════════
  GRAND ORCHESTRATOR: LLM + MCP + A2A
════════════════════════════════════════════════════════════

💬 User: Search for flights from SFO to Tokyo on 2026-10-01


▶ Step 1: Connecting to MCP server for tools
───────────────────────────────────────────────────────
  ✅ MCP server connected
  🔧 MCP tools: ['search_flights', 'search_hotels', 'check_calendar', 'estimate_daily_expenses', 'calculate_trip_budget']
  🤖 A2A agents: currency-agent, finance-agent
  📊 Total tools available to LLM: 6

▶ Step 2: Reading MCP resources for context
───────────────────────────────────────────────────────
  📄 Loaded: user preferences + Tokyo tips

▶ Step 3: Starting agentic loop (LLM decides tools vs agents)
───────────────────────────────────────────────────────

  🔄 Iteration 1
  ─────────────────────────────────────────────
  🔧 MCP TOOL → search_flights
     Args: {"origin": "SFO", "destinatio

In [77]:
# Cell A31: Test 2 — Budget analysis → expects A2A delegation to finance-agent

print("=" * 60)
print("  TEST 2: Budget analysis → expects A2A finance-agent")
print("=" * 60)

answer, stats = await grand_orchestrator(
    "I have a budget of 450000 JPY. Convert that to USD and tell me "
    "if a 5-day Tokyo trip with $820 flights and $950 hotel is feasible."
)

print(f"\n{'═' * 55}")
print("📋 FINAL ANSWER:")
print(f"{'═' * 55}")
print(answer)

print(f"\n📊 Stats: MCP calls={stats['mcp_tool_calls']}, A2A delegations={stats['a2a_delegations']}")
print(f"   MCP tools: {stats['mcp_tools_used']}")
print(f"   A2A agents: {stats['agents_used']}")

  TEST 2: Budget analysis → expects A2A finance-agent

════════════════════════════════════════════════════════════
  GRAND ORCHESTRATOR: LLM + MCP + A2A
════════════════════════════════════════════════════════════

💬 User: I have a budget of 450000 JPY. Convert that to USD and tell me if a 5-day Tokyo trip with $820 flights and $950 hotel is feasible.


▶ Step 1: Connecting to MCP server for tools
───────────────────────────────────────────────────────
  ✅ MCP server connected
  🔧 MCP tools: ['search_flights', 'search_hotels', 'check_calendar', 'estimate_daily_expenses', 'calculate_trip_budget']
  🤖 A2A agents: currency-agent, finance-agent
  📊 Total tools available to LLM: 6

▶ Step 2: Reading MCP resources for context
───────────────────────────────────────────────────────
  📄 Loaded: user preferences + Tokyo tips

▶ Step 3: Starting agentic loop (LLM decides tools vs agents)
───────────────────────────────────────────────────────

  🔄 Iteration 1
  ─────────────────────────────────

/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  🔧 MCP TOOL → estimate_daily_expenses
     Args: {"city": "Tokyo", "style": "moderate"}
  📥 Tool result (166 chars)

  🔄 Iteration 3
  ─────────────────────────────────────────────
  🔧 MCP TOOL → calculate_trip_budget
     Args: {"flight_cost": 820, "hotel_cost": 950, "daily_expense": 135, "num_days": 5, "to
  📥 Tool result (232 chars)

  🔄 Iteration 4
  ─────────────────────────────────────────────

  ✅ LLM finished!

═══════════════════════════════════════════════════════
📋 FINAL ANSWER:
═══════════════════════════════════════════════════════
450,000 JPY ≈ **$3,010.03 USD** (using ~**1 JPY = $0.0067**).

### 5-day Tokyo trip feasibility (moderate daily spend)
Assuming **$820 flights** + **$950 hotel** and estimated Tokyo daily expenses of **$135/day**:

- Flights: **$820**
- Hotel: **$950**
- Daily expenses: **$135 × 5 = $675**
- **Total trip cost: $2,445**

### Verdict
With a budget of **$3,010.03**, this trip is **feasible**.

- **Remaining budget:** **$3,010.03 − $2,445 = $565.03

In [78]:
# Cell A32: Test 3 — The FULL comprehensive query
# LLM should use BOTH MCP tools AND A2A agents

print("=" * 60)
print("  TEST 3: FULL QUERY — MCP tools + A2A agents together!")
print("=" * 60)

FULL_QUERY = """
Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.

Please:
1. Check my calendar for any conflicts in early October 2026
2. Search for flights from SFO to Tokyo around Oct 1
3. Search for boutique hotels in Tokyo for 5 nights
4. Get a professional budget feasibility analysis considering
   flights, hotel, and daily expenses
5. Also tell me how much my $3000 budget is in Japanese Yen
6. Give me a complete plan with all the details
"""

start_time = time.time()
answer, stats = await grand_orchestrator(FULL_QUERY)
total_time = time.time() - start_time

print(f"\n{'═' * 55}")
print("📋 COMPLETE TRAVEL PLAN:")
print(f"{'═' * 55}")
print(answer)

  TEST 3: FULL QUERY — MCP tools + A2A agents together!

════════════════════════════════════════════════════════════
  GRAND ORCHESTRATOR: LLM + MCP + A2A
════════════════════════════════════════════════════════════

💬 User: 
Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.

Please:
1. Check my calendar for any conflicts in early October 2026
2. Search for flights from SFO to Tokyo around Oct 1
3. Search for boutique hotels in Tokyo for 5 nights
4. Get a professional budget feasibility analysis considering 
   flights, hotel, and daily expenses
5. Also tell me how much my $3000 budget is in Japanese Yen
6. Give me a complete plan with all the details



▶ Step 1: Connecting to MCP server for tools
───────────────────────────────────────────────────────
  ✅ MCP server connected
  🔧 MCP tools: ['search_flights', 'search_hotels', 'check_calendar', 'estimate_daily_expenses', 'calculate_trip_budget']
  🤖 A2A agents: currency-agent, finance-a

/tmp/ipython-input-3925901400.py:123: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)


  🤖 A2A DELEGATION → finance-agent
     Message: "Provide a professional feasibility analysis for a 5-day Tokyo trip (5 nights) wi..."
  📤 Sending: "Provide a professional feasibility analysis for a 5-day Tokyo trip (5 nights) with a $3000 total budget. Use flight options $620-$910, boutique hotels $875-$1100 total for 5 nights, daily expenses $135/day for 5 days. Include risk factors (exchange rate, taxes/fees, transit from airport), and recommendations to stay within budget."
  📥 Response received!
  📥 Agent response (3780 chars)

  🔄 Iteration 3
  ─────────────────────────────────────────────
  🔧 MCP TOOL → calculate_trip_budget
     Args: {"flight_cost": 820, "hotel_cost": 950, "daily_expense": 135, "num_days": 5, "to
  📥 Tool result (219 chars)

  🔄 Iteration 4
  ─────────────────────────────────────────────

  ✅ LLM finished!

═══════════════════════════════════════════════════════
📋 COMPLETE TRAVEL PLAN:
═══════════════════════════════════════════════════════
## 1) Calendar conf

In [79]:
# Cell A33: Display the execution statistics

print_header("📊 GRAND FINALE — Execution Statistics")

print(f"""
┌─ Execution Summary ─────────────────────────────
│
│  ⏱️  Total time:        {total_time:.1f}s
│  🔄 Iterations:        {stats['iterations']}
│
│  🔧 MCP Tool Calls:    {stats['mcp_tool_calls']}
│     Tools used:        {', '.join(stats['mcp_tools_used']) if stats['mcp_tools_used'] else 'none'}
│
│  🤖 A2A Delegations:   {stats['a2a_delegations']}
│     Agents used:       {', '.join(stats['agents_used']) if stats['agents_used'] else 'none'}
│
│  📊 Total actions:     {stats['mcp_tool_calls'] + stats['a2a_delegations']}
│
└──────────────────────────────────────────────────
""")


════════════════════════════════════════════════════════════
  📊 GRAND FINALE — Execution Statistics
════════════════════════════════════════════════════════════

┌─ Execution Summary ─────────────────────────────
│
│  ⏱️  Total time:        55.1s
│  🔄 Iterations:        4
│
│  🔧 MCP Tool Calls:    6
│     Tools used:        search_hotels, check_calendar, search_flights, calculate_trip_budget, estimate_daily_expenses
│
│  🤖 A2A Delegations:   2
│     Agents used:       finance-agent, currency-agent
│
│  📊 Total actions:     8
│
└──────────────────────────────────────────────────



In [80]:
# Cell A34: The complete architecture we built

print_header("🏗️ COMPLETE ARCHITECTURE")

print("""
                         ┌─────────────┐
                         │    USER     │
                         │ "Plan trip" │
                         └──────┬──────┘
                                │
                 ┌──────────────▼──────────────┐
                 │     🧠 LLM ORCHESTRATOR     │
                 │        (GPT-4o-mini)         │
                 │                              │
                 │  Reads: MCP Resources        │
                 │    • user preferences        │
                 │    • Tokyo travel tips        │
                 │                              │
                 │  Decides dynamically:         │
                 │    → Use MCP tool? (direct)  │
                 │    → Delegate to A2A agent?  │
                 └───┬─────────────────┬────────┘
                     │                 │
            MCP Tools│                 │A2A Protocol
        (stdio/local)│                 │(HTTP/JSON-RPC)
                     │                 │
         ┌───────────▼──┐    ┌────────▼──────────────┐
         │  MCP Server  │    │    A2A Agents          │
         │              │    │                        │
         │ 🔧 Tools:    │    │ 🤖 currency-agent     │
         │ •flights     │    │    (port 5001)         │
         │ •hotels      │    │    Simple logic        │
         │ •calendar    │    │    No LLM inside       │
         │ •expenses    │    │                        │
         │ •budget calc │    │ 🤖 finance-agent       │
         │              │    │    (port 5002)         │
         │ 📄 Resources:│    │    LLM + tools inside  │
         │ •preferences │    │    AI-powered analysis  │
         │ •travel tips │    │                        │
         └──────────────┘    └────────────────────────┘
""")


════════════════════════════════════════════════════════════
  🏗️ COMPLETE ARCHITECTURE
════════════════════════════════════════════════════════════

                         ┌─────────────┐
                         │    USER     │
                         │ "Plan trip" │
                         └──────┬──────┘
                                │
                 ┌──────────────▼──────────────┐
                 │     🧠 LLM ORCHESTRATOR     │
                 │        (GPT-4o-mini)         │
                 │                              │
                 │  Reads: MCP Resources        │
                 │    • user preferences        │
                 │    • Tokyo travel tips        │
                 │                              │
                 │  Decides dynamically:         │
                 │    → Use MCP tool? (direct)  │
                 │    → Delegate to A2A agent?  │
                 └───┬─────────────────┬────────┘
                     │                 │
           

In [81]:
# Cell A35: Key takeaways from the entire demo

print_header("🔑 KEY TAKEAWAYS")

print("""
  1️⃣  LLM ALONE is smart but BLIND
     → Hallucinated prices, no real actions
     → Part 1 demonstrated this clearly

  2️⃣  MCP gives an agent TOOLS and CONTEXT
     → "USB-C for AI" — standard tool integration
     → Tools (actions) + Resources (data) + Prompts (templates)
     → Part 2 (Kiwi.com) & Part 3 (custom) showed this

  3️⃣  A2A lets agents DISCOVER and COLLABORATE
     → "HTTP for Agents" — standard agent communication
     → Agent Cards (discovery) + Tasks (work) + Messages (communication)
     → Part 4-6 showed individual and multi-agent scenarios

  4️⃣  MCP and A2A are COMPLEMENTARY
     → MCP = vertical (agent ↔ tools) — WITHIN an agent
     → A2A = horizontal (agent ↔ agent) — BETWEEN agents
     → Part 7 showed them working TOGETHER

  5️⃣  The LLM is the BRAIN that decides
     → When to use MCP tools (direct action)
     → When to delegate to A2A agents (specialized tasks)
     → We wrote ZERO hardcoded orchestration logic

  6️⃣  A2A agents are OPAQUE from outside
     → Currency agent: simple regex (no LLM)
     → Finance agent: full LLM + tools inside
     → From outside, they look identical (same protocol)

  7️⃣  Both are OPEN STANDARDS
     → MCP: by Anthropic (open source)
     → A2A: by Google (open source)
     → Not locked to any vendor
""")

print(f"""
{'═' * 55}
  🎓 ANALOGY: Think of a HOSPITAL

  🏥 Hospital         = Your AI Application
  👨‍⚕️ Each Doctor       = An Agent (A2A)
  🔧 Their instruments = MCP Tools
  📋 Patient records   = MCP Resources
  📇 Hospital directory= Agent Cards
  📝 Referral system   = A2A Tasks
  🧠 Medical knowledge = LLM
{'═' * 55}
""")


════════════════════════════════════════════════════════════
  🔑 KEY TAKEAWAYS
════════════════════════════════════════════════════════════

  1️⃣  LLM ALONE is smart but BLIND
     → Hallucinated prices, no real actions
     → Part 1 demonstrated this clearly

  2️⃣  MCP gives an agent TOOLS and CONTEXT
     → "USB-C for AI" — standard tool integration
     → Tools (actions) + Resources (data) + Prompts (templates)
     → Part 2 (Kiwi.com) & Part 3 (custom) showed this

  3️⃣  A2A lets agents DISCOVER and COLLABORATE
     → "HTTP for Agents" — standard agent communication
     → Agent Cards (discovery) + Tasks (work) + Messages (communication)
     → Part 4-6 showed individual and multi-agent scenarios

  4️⃣  MCP and A2A are COMPLEMENTARY
     → MCP = vertical (agent ↔ tools) — WITHIN an agent
     → A2A = horizontal (agent ↔ agent) — BETWEEN agents
     → Part 7 showed them working TOGETHER

  5️⃣  The LLM is the BRAIN that decides
     → When to use MCP tools (direct action)
     

In [82]:
# Cell A36: Grand comparison table across all parts

print_header("🏆 GRAND COMPARISON: The Full Journey")

print("""
┌──────────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│                  │ Part 1     │ Part 2      │ Part 3      │ Part 7       │
│                  │ Raw LLM    │ Kiwi MCP    │ Custom MCP  │ MCP + A2A    │
├──────────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│ Real data        │ ❌         │ ✅ flights  │ ✅ all      │ ✅ all       │
│ MCP tools        │ 0          │ External    │ 5 custom    │ 5 custom     │
│ MCP resources    │ 0          │ 0           │ 2           │ 2            │
│ A2A agents       │ 0          │ 0           │ 0           │ 2 agents     │
│ Calendar check   │ ❌         │ ❌          │ ✅          │ ✅           │
│ Budget analysis  │ ❌         │ ❌          │ ✅ basic    │ ✅ AI-powered│
│ Currency convert │ ❌         │ ❌          │ ❌          │ ✅ via A2A   │
│ Personalized     │ ❌         │ ❌          │ ✅          │ ✅           │
│ Multi-agent      │ ❌         │ ❌          │ ❌          │ ✅           │
├──────────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│ Architecture     │ LLM only   │ LLM + MCP   │ LLM + MCP   │ LLM+MCP+A2A │
└──────────────────┴────────────┴─────────────┴─────────────┴──────────────┘
""")


════════════════════════════════════════════════════════════
  🏆 GRAND COMPARISON: The Full Journey
════════════════════════════════════════════════════════════

┌──────────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│                  │ Part 1     │ Part 2      │ Part 3      │ Part 7       │
│                  │ Raw LLM    │ Kiwi MCP    │ Custom MCP  │ MCP + A2A    │
├──────────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│ Real data        │ ❌         │ ✅ flights  │ ✅ all      │ ✅ all       │
│ MCP tools        │ 0          │ External    │ 5 custom    │ 5 custom     │
│ MCP resources    │ 0          │ 0           │ 2           │ 2            │
│ A2A agents       │ 0          │ 0           │ 0           │ 2 agents     │
│ Calendar check   │ ❌         │ ❌          │ ✅          │ ✅           │
│ Budget analysis  │ ❌         │ ❌          │ ✅ basic    │ ✅ AI-powered│
│ Currency convert │ ❌         │ ❌          │ ❌          │ ✅ via A2A   │
│ Pers

# 🎉 Demo Complete!

## What we built and demonstrated:

| Part | Component | Key Lesson |
|------|-----------|------------|
| **1** | Raw LLM | LLMs hallucinate without tools |
| **2** | LLM + Kiwi MCP | External MCP servers give real data |
| **3** | LLM + Custom MCP | Build your own tools, resources, prompts |
| **4** | Simple A2A Agent | Agents expose capabilities via Agent Cards |
| **5** | A2A Agent + LLM | Agents can be smart internally (opaque) |
| **6** | Multi-Agent Discovery | LLM orchestrator discovers and delegates |
| **7** | **Grand Finale** | **MCP + A2A + LLM = Complete agentic system** |

## The two protocols:
- **MCP** = Plug tools/data into an agent (vertical)
- **A2A** = Let agents collaborate (horizontal)
- **Together** = Scalable, modular AI ecosystem

## The LLM's role:
- **Brain** that decides what to do
- **Zero hardcoded logic** — LLM figures out the workflow
- Dynamically chooses between direct tools and agent delegation

In [ ]:
# Cell A37: Cleanup — stop all background agent servers

stop_all_agents()
print("\n✅ Demo complete! All agents stopped.")
print("📁 Files created:")
print("   mcp_server/travel_server.py  — Custom MCP server")
print("   a2a_agents/currency_agent.py — Currency A2A agent")
print("   a2a_agents/finance_agent.py  — Finance A2A agent (LLM-powered)")